In [228]:
# Jenny branch
import numpy as np

import datetime

import tensorflow as tf 

from sklearn import linear_model, ensemble, tree, svm, naive_bayes

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold


BORDER = "===================================================================================="

In [189]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [190]:
def WritePredictionOut(modelName, prediction):
    print(prediction)
    print("prediction shape is: {}".format(prediction.shape))

    output = "Id,Prediction\n"
    for i in range(prediction.shape[0]):
        output = output + ("{0},{1}\n".format(i + 1, prediction[i].astype(int)))

        
    now = datetime.datetime.now();
        
    filename = "{}_{}.{}.{}_{}.{}.{}_predictions.csv".format(modelName, now.year, now.month, now.day, now.hour, now.minute, now.second)
        
    file = open(filename,'w') 
    file.write(output)
    file.close()

In [191]:
def ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    # Layer set up.
    model = Sequential()
    
    model.add(Dense(1024, input_shape=(n_words, )))
    model.add(Activation('relu'))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=512, epochs=2, verbose=verbose, validation_split=0.1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [192]:
def ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    
    # For 1D Convolution, change dimension of input 
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
        
    ## Create your own model here given the constraints in the problem
    model = Sequential()

    model.add(Conv1D(64, 3, padding="same", input_shape=(n_words, 1, )))
    
    model.add(Conv1D(32, 3, padding="same"))
    
    model.add(Conv1D(16, 3, padding="same"))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(120))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    ## Once you one-hot encode the data labels, the line below should be predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=128, epochs=2, verbose=verbose, validation_split=0.1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [193]:
def ClassifyWithLogisticRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLOGISTIC REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [194]:
def ClassifyWithLinearRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLINEAR REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LinearRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [195]:
def ClassifyWithAdaBoost(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """

    if (verbose == 1):
        print('\n{}\nADABOOST\n{}\n'.format(BORDER, BORDER))
    
    model = ensemble.AdaBoostClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [196]:
def ClassifyWithDecisionTree(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nDECISION TREE\n{}\n'.format(BORDER, BORDER))
        
    model = tree.DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [197]:
def ClassifyWithRandomForest(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nRANDOM FOREST\n{}\n'.format(BORDER, BORDER))
        
    model = ensemble.RandomForestClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [198]:
def ClassifyWithSVM(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nSVM\n{}\n'.format(BORDER, BORDER))
        
    model = svm.LinearSVC(verbose=verbose)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [199]:
def ClassifyWithBernoulliNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nBernoulli Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.BernoulliNB(alpha=.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [200]:
def ClassifyWithMultinomialNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\n Multinomial Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.MultinomialNB(alpha=.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [201]:
def CrossValidation_NeuralNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
     # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # perform 5-fold validation
    for i in range(0,5):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_test)
        
        # Compute accuracy.
        val_acc = model.evaluate(X_train[vali], Y[vali], verbose=verbose)
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithNeuralNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0


In [202]:
def CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    
    # For 1D Convolution, change dimension of input 
    X = np.expand_dims(X_train, axis=2)
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini],X_test)
                
        # Compute accuracy
        val_acc = model.evaluate(X[vali], Y[vali], verbose=verbose)

        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithConvolutionNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0

In [203]:
def CrossValidation_sklearn(modelFunction, X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    
    # perform 5-fold validation
    for i in range(0,5):
        traini, vali = inds[i]
        model, prediction, accuracy = modelFunction(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        
        # Compute accuracy.
        val_acc = model.score(X_train[vali], Y_train[vali])
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: " + modelFunction.__name__)
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))
    
    return 0


In [233]:
def Create_NeuralNetwork_model(neurons=1, verbose=0):
    num_classes = 2
    n_words = 1000
    
    # Layer set up.
    model = Sequential()
    
    model.add(Dense(1024, input_shape=(n_words, )))
    model.add(Activation('relu'))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    # Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()
        
    # Compile model
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

In [230]:
def GridSearch_NeuralNetwork(X_train, Y_train, verbose=0):

    model = KerasClassifier(build_fn=Create_NeuralNetwork_model, epochs=2, batch_size=128, verbose=verbose)
 
    # Define the grid search parameters
    neurons = [2 ** x for x in range(0, 10)]
    param_grid = dict(neurons=neurons)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result =  grid.fit(X_train, Y_train)
    
    print(BORDER)
    print("GridSearch: Create_NeuralNetwork_model")
    print(BORDER)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    

In [206]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:, 0]

In [207]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [ ]:
# Using Grid Search
GridSearch_NeuralNetwork(X_train, Y_train)

neurons [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
parm_grid {'neurons': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]}


In [ ]:
# Using Validation   
# CrossValidation_NeuralNetwork(X_train, Y_train, X_test, 5)
# CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithLinearRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithLogisticRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithAdaBoost, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithDecisionTree, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithRandomForest, X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithSVM, X_train, Y_train, X_test, 5, verbose=1)

# CrossValidation_sklearn(ClassifyWithBernoulliNB, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithMultinomialNB, X_train, Y_train, X_test, 5)

In [ ]:
_, prediction, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=0)
WritePredictionOut('NeuralNetwork', prediction)